# Convection-Diffusion-Reaction Model for Forest Fire

In [30]:
import numpy as np
import matplotlib.pyplot as plt
from utils import weno5_left, weno5_right

In [31]:
# Parameters and such
R = 1.987207  # universal gas constant in cal/(K*mol)
E_A = 20  # activation energy in kcal/mol
U_inf = 303  # ambient temperature in K
eps = 0.03
A = 1e9
rho = 100  # kg/m^3
C = 1  # kJ /(kg*K)
k = 1  # W/(m*K)
H = 300  # kJ/kg
t0 = 8986.8
l0 = 0.2998
L = 10  # Omega = [0, L] x [0, L]
kappa = 0.01  # inverse of conductivity coefficient
q_react = 1

u_pc = 0  # not sure on this one
C_fl = 0.12

In [32]:
alpha = lambda x: 1e-3 * np.ones_like(x)
c = lambda u: np.where(u >= u_pc, 1, 0)

K = lambda u: kappa * (1 + eps * u) + 1  
zeta = lambda u: c(u) * np.exp(u / (1 + eps * u))
f = lambda u, v, x: v * zeta(u) - alpha(x)
g = lambda u, v: -eps * v * zeta(u) / q_react

In [33]:
M = 100  # number of spatial points

dx = L / M
x = y = np.linspace(dx/2, L - dx/2, M)
X, Y = np.meshgrid(x, y, indexing='ij')  # (X[i, j], Y[i, j]) = (x[i], y[j])

In [40]:
import numpy as np
from scipy.sparse.linalg import LinearOperator, bicgstab

#------------------------------------------------------------------------------
# Utility: pad array with ghost cells enforcing zero-flux (Neumann) BCs
#------------------------------------------------------------------------------
def pad_with_ghosts(arr, n_ghosts=1):
    return np.pad(arr, pad_width=n_ghosts, mode='edge')

#------------------------------------------------------------------------------
# WENO5: reconstruct a single interface value from 5-point stencil
#------------------------------------------------------------------------------
def weno5_reconstruct_1d(v):
    v0, v1, v2, v3, v4 = v
    q0 = (2*v0 - 7*v1 + 11*v2) / 6.0
    q1 = (-v1 + 5*v2 + 2*v3) / 6.0
    q2 = (2*v2 + 5*v3 - v4) / 6.0
    beta0 = (13/12)*(v0 - 2*v1 + v2)**2 + 0.25*(v0 - 4*v1 + 3*v2)**2
    beta1 = (13/12)*(v1 - 2*v2 + v3)**2 + 0.25*(v1 - v3)**2
    beta2 = (13/12)*(v2 - 2*v3 + v4)**2 + 0.25*(3*v2 - 4*v3 + v4)**2
    alpha0 = 0.1 / ( (1e-6 + beta0)**2 )
    alpha1 = 0.6 / ( (1e-6 + beta1)**2 )
    alpha2 = 0.3 / ( (1e-6 + beta2)**2 )
    w0 = alpha0 / (alpha0 + alpha1 + alpha2)
    w1 = alpha1 / (alpha0 + alpha1 + alpha2)
    w2 = alpha2 / (alpha0 + alpha1 + alpha2)
    return w0*q0 + w1*q1 + w2*q2

#------------------------------------------------------------------------------
# Convection operator C(u) in 2D with separate x/y fluxes
#------------------------------------------------------------------------------
def compute_C(u, flux_x, flux_x_prime, flux_y, flux_y_prime, dx):
    nx, ny = u.shape
    n_ghost = 3
    u_pad = pad_with_ghosts(u, n_ghost)
    fx_pad = flux_x(u_pad)
    fy_pad = flux_y(u_pad)
    alpha_x = np.max(np.abs(flux_x_prime(u_pad)))
    alpha_y = np.max(np.abs(flux_y_prime(u_pad)))
    fx_plus  = 0.5*(fx_pad + alpha_x*u_pad)
    fx_minus = 0.5*(fx_pad - alpha_x*u_pad)
    fy_plus  = 0.5*(fy_pad + alpha_y*u_pad)
    fy_minus = 0.5*(fy_pad - alpha_y*u_pad)

    C = np.zeros_like(u)
    for i in range(nx):
        for j in range(ny):
            ip, jp = i + n_ghost, j + n_ghost
            # x-direction flux differences
            fhat_ip = (
                weno5_reconstruct_1d(fx_plus[ip-2:ip+3, jp]) +
                weno5_reconstruct_1d(fx_minus[ip-2:ip+3, jp][::-1])[::-1]
            )
            fhat_im = (
                weno5_reconstruct_1d(fx_plus[ip-3:ip+2, jp]) +
                weno5_reconstruct_1d(fx_minus[ip-3:ip+2, jp][::-1])[::-1]
            )
            # y-direction flux differences
            ghat_jp = (
                weno5_reconstruct_1d(fy_plus[ip, jp-2:jp+3]) +
                weno5_reconstruct_1d(fy_minus[ip, jp-2:jp+3][::-1])[::-1]
            )
            ghat_jm = (
                weno5_reconstruct_1d(fy_plus[ip, jp-3:jp+2]) +
                weno5_reconstruct_1d(fy_minus[ip, jp-3:jp+2][::-1])[::-1]
            )
            C[i, j] = -((fhat_ip - fhat_im) + (ghat_jp - ghat_jm)) / dx
    return C

#------------------------------------------------------------------------------
# Diffusion operator D(u)u
#------------------------------------------------------------------------------
def compute_Du(K, u, dx):
    nx, ny = u.shape
    n_ghost = 1
    u_pad = pad_with_ghosts(u, n_ghost)
    K_pad = pad_with_ghosts(K, n_ghost)

    Du = np.zeros_like(u)
    for i in range(nx):
        for j in range(ny):
            ip, jp = i + n_ghost, j + n_ghost
            K_i  = K_pad[ip,   jp]
            K_im = K_pad[ip-1, jp]
            K_ip = K_pad[ip+1, jp]
            u_im = u_pad[ip-1, jp]
            u_i  = u_pad[ip,   jp]
            u_ip = u_pad[ip+1, jp]
            term_x = ((K_i+K_im)*u_im - (K_im+2*K_i+K_ip)*u_i + (K_i+K_ip)*u_ip) / dx**2
            K_jm = K_pad[ip, jp-1]
            K_jp = K_pad[ip, jp+1]
            u_jm = u_pad[ip, jp-1]
            u_jp = u_pad[ip, jp+1]
            term_y = ((K_i+K_jm)*u_jm - (K_jm+2*K_i+K_jp)*u_i + (K_i+K_jp)*u_jp) / dx**2
            Du[i, j] = term_x + term_y
    return Du

#------------------------------------------------------------------------------
# H-LDIRK3(2,2,2) IMEX-RK time step with separate fluxes
#------------------------------------------------------------------------------
def imex_lidirk_step(u_n, v_n, dt, K, A_field, zeta,
                     flux_x, flux_x_prime, flux_y, flux_y_prime,
                     dx, epsilon, q_react):
    gamma = (3 + np.sqrt(3)) / 6.0
    A_exp = np.array([[0.0, 0.0, 0.0], [1.0, 0.0, 0.0], [0.5, 0.5, 0.0]])
    A_imp = np.array([[gamma,       0.0,    0.0],
                      [1-gamma,   gamma,    0.0],
                      [0.5,      0.5-gamma, gamma]])
    b = np.array([1/6, 2/3, 1/6])
    s = 3

    nx, ny = u_n.shape
    N = nx * ny
    K_u = [None]*s
    K_v = [None]*s

    for i in range(s):
        # stage prep
        u_star = u_n.copy()
        u_hat  = u_n.copy()
        v_hat  = v_n.copy()
        for j in range(i):
            u_star += dt * A_exp[i,j] * K_u[j]
            u_hat  += dt * A_imp[i,j] * K_u[j]
            v_hat  += dt * A_imp[i,j] * K_v[j]

        # explicit v-stage
        zeta_star = zeta(u_star)
        K_v_i = -(epsilon/q_react) * v_hat * zeta_star \
                / (1 + (epsilon/q_react)*dt*A_imp[i,i]*zeta_star)
        K_v[i] = K_v_i

        # explicit C and Du
        C_star   = compute_C(u_star, flux_x, flux_x_prime, flux_y, flux_y_prime, dx)
        Du_u_hat = compute_Du(K, u_hat, dx)

        # build and solve linear system for K_u
        def matvec(x):
            X = x.reshape((nx, ny))
            return (compute_Du(K, X, dx) - A_field*X).flatten()
        L = LinearOperator((N,N), matvec=lambda x: x - dt*A_imp[i,i]*matvec(x))
        rhs = (C_star + Du_u_hat - A_field*u_hat \
               - (v_hat + dt*A_imp[i,i]*K_v_i)*zeta_star).flatten()
        Ku_i_vec, info = bicgstab(L, rhs)
        if info != 0:
            raise RuntimeError(f"Linear solve failed at stage {i}, info={info}")
        K_u[i] = Ku_i_vec.reshape((nx, ny))

    # combine stages
    u_np1 = u_n.copy()
    v_np1 = v_n.copy()
    for i in range(s):
        u_np1 += dt * b[i] * K_u[i]
        v_np1 += dt * b[i] * K_v[i]

    return u_np1, v_np1


In [ ]:
# --- grid + domain ---
M = 100
x0, x1 = 0.0, 1.0
y0, y1 = 0.0, 1.0
x = np.linspace(x0, x1, M)
y = np.linspace(y0, y1, M)
X, Y = np.meshgrid(x, y, indexing='ij')
dx = x[1] - x[0]

U = 2.0

# now flux_x(u) returns an array same shape as u
flux_x       = lambda u: U * u
# and its derivative is U everywhere
flux_x_prime = lambda u: U * np.ones_like(u)

# no y‑wind
flux_y       = lambda u: 0 * u
flux_y_prime = lambda u: 0 * u


# --- time‐stepping ---
t, dt, t_end = 0.0, 0.01, 0.1
steps = int((t_end - t)/dt)

# --- initial data + BCs ---
u = np.zeros((M,M))
v = np.zeros((M,M))
u[0,:] = u[-1,:] = u[:,0] = u[:,-1] = 1.0
v[0,:] = v[-1,:] = v[:,0] = v[:,-1] = 1.0

# --- time‐loop ---
for n in range(steps):
    # build diffusion + reaction‐field
    K       = kappa*(1 + eps*u) + 1.0
    A_field = alpha(X)             # same shape as u
    
    u, v = imex_lidirk_step(
        u, v, dt,
        K, A_field, zeta,
        flux_x, flux_x_prime,
        flux_y, flux_y_prime,
        dx, eps, q_react
        )
    t += dt
    print(f"Step {n+1}/{steps}: t = {t:.3f}")


# --- plot final u ---
plt.figure(figsize=(6,5))
plt.pcolormesh(X, Y, u, shading='auto')
plt.colorbar(label='u')
plt.title('u at t = %.3f' % t)
plt.xlabel('x'); plt.ylabel('y')
plt.show()


IndexError: invalid index to scalar variable.